In [198]:
from mnist.load_mnist import MNIST

from nn.model import Sequential
from nn.activations import relu, tanh, sigmoid
from nn.layers import Dense
from nn.loss import MeanSquaredError

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [119]:
data = MNIST(test_split=0.25)
train_data, test_data = data.dataset
X_train, y_train = train_data
X_test, y_test = test_data

print('Train:', X_train.shape, y_train.shape)
print('Test:', X_test.shape, y_test.shape)

Train: (31500, 784) (31500,)
Test: (10500, 784) (10500,)


In [120]:
X_train[0].shape

(784,)

In [122]:
nn.info()

model: Sequential
Dense | (128,784) | relu
Dense | (10,128) | tanh


In [144]:
from typing import Union


def categorical_encoding(array: Union[np.ndarray, list]) -> np.ndarray:
    len_ = np.max(array) + 1
    initial_shape = array.flatten().shape
    new_shape = (initial_shape[0], len_)
    
    encoded_array = np.empty(new_shape)
    
    for value, row in zip(array, range(initial_shape[0])):
        sample = np.zeros(len_)
        sample[value] = 1.
        
        encoded_array[row] = np.expand_dims(sample, axis=0)
        
    return encoded_array

In [ ]:
from typing import List


def transform_input_data(array: np.ndarray) -> np.ndarray:
    return np.array(
        list(
            map(
                lambda sample: sample.reshape(-1, 1), X_train
                )
            )
        )

In [145]:
a = categorical_encoding(y_train)

7


7

In [170]:
nn = Sequential(input_shape=(1, 784),
                layers=[
                    Dense(128, activation=relu),
                    Dense(10, activation=tanh)],
                loss=MeanSquaredError())

In [202]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [204]:
X_train_ = list(map(lambda sample: sample.reshape(-1, 1), X_train))
y_train_ = categorical_encoding(y_train)
y_train_ = list(map(lambda sample: sample.reshape(-1, 1), y_train_))

In [219]:
a = np.array(list(map(lambda x: x**2, range(1, 5))))
a

array([ 1,  4,  9, 16])

In [205]:
e = nn.forward(X_train_[0])
e

array([[0.08721562],
       [0.7271749 ],
       [0.32179857],
       [0.46687585],
       [0.113234  ],
       [0.68713775],
       [0.66063377],
       [0.28242324],
       [0.14282817],
       [0.63430047]])

In [206]:
y_train_[0]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.]])

In [207]:
nn.fit(X_train_, y_train_, epochs=2)

Start training for 2 epochs
Epoch : 0 - Loss: 0.013763239177670018
Epoch : 1 - Loss: 0.011125425964843696


In [214]:
print(y_train[12])
a = nn.predict(X_train_[12])


1


array([ 0.01798885,  0.83850607,  0.32513576, -0.04462528,  0.05465042,
       -0.02596323, -0.00918569, -0.08132664,  0.0292984 , -0.05120131])